In [26]:
import matplotlib
import numpy as np
from IPython.display import HTML, display


def visualize_sentences(sentences, weight_lists, cmap_name="OrRd"):
    """
    Visualize multiple sentences with token weights in a list format, each with a white background.

    Args:
    - sentences (list of list of str): A list of sentences, where each sentence is a list of tokens.
    - weight_lists (list of list of float): A list of weight lists, one for each sentence's tokens.
    - cmap_name (str): Name of the matplotlib colormap to use.

    Returns:
    - None: Displays the highlighted sentences.
    """
    # Get colormap
    cmap = matplotlib.colormaps[cmap_name]
    
    # Build HTML for each sentence
    sentence_html_list = []
    for tokens, weights in zip(sentences, weight_lists):
        weights = np.array(weights)

        # Normalize weights to [0, 1]
        norm_weights = (weights - min(weights)) / (max(weights) - min(weights))
        
        # Convert weights to colors
        colors = [matplotlib.colors.rgb2hex(cmap(w)) for w in norm_weights]
        
        # Create highlighted text for the sentence
        highlighted_sentence = " ".join(
            f'<span style="background-color:{color}; padding:0px;  font-weight:bold; color:black;">{token}</span>'
            for token, color in zip(tokens, colors)
        )
        sentence_html_list.append(f"<li>{highlighted_sentence}</li>")
    
    # Combine all sentences into a single container with a white background
    html_content = f"""
    <div style='font-family:monospace; background-color:white; padding:10px;'>
        <ul style='list-style-type:none; padding:1px; margin:1px;'>
            {''.join(sentence_html_list)}
        </ul>
    </div>
    """
    display(HTML(html_content))


# Example usage
sentences = [
    ["This", "is", "a", "test", "."],
    ["Another", "example", "sentence", "here", "."],
    ["Visualizing", "multiple", "sentences", "is", "easy", "!"]
]
weight_lists = [
    [0.1, 0.2, 0.5, 0.8, 0.3],
    [0.3, 0.6, 0.9, 0.4, 0.2],
    [0.5, 0.3, 0.8, 0.6, 0.7, 0.9]
]
visualize_sentences(sentences, weight_lists)
